# Yahoo Finance Agent

This notebook goes over how to use yahoo_finance_news tool with an agent and how to retrieve additional financial information from yahoo finance API

In [182]:
# %pip install --upgrade --quiet  yfinance==0.2.44 langchain-community==0.3.1 langchain-openai==0.2.1 langchain-core==0.3.8 langchain==0.3. pandas pydantic==2.9.1

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [197]:
%load_ext watermark
%watermark --iversions

yfinance           : 0.2.44
langchain          : 0.3.1
pandas             : 2.2.3
langchain_openai   : 0.2.1
langchain_community: 0.3.1
json               : 2.0.9
langchain_core     : 0.3.8
pydantic           : 2.9.1



## Setting Up

In [183]:
# Standard library imports
import os
from datetime import datetime

# Third-party imports
from dotenv import load_dotenv
import pandas as pd
import yfinance as yf
import json

# LangChain imports
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_core.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI
from langchain import hub
from langchain_core.documents import Document

# Load environment variables
load_dotenv(r"C:\Lang Graph\Shared Notebook\credentials_sf.env", override=True)


True

# Yahoo Finance News

## Use AgentExecutor

Here we will be using basic React Prompt Template

In [184]:
template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat 1 times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

# llm 
llm = AzureChatOpenAI(azure_deployment='Agent', temperature= 0)

# tools
tools = [YahooFinanceNewsTool()]

# create an agent 
agent = create_react_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent = agent, tools = tools)

In [185]:
agent_executor.invoke({'input' : 'What happend with Morgan stanley stock yesterday'})

{'input': 'What happend with Morgan stanley stock yesterday',
 'output': "Morgan Stanley's stock increased by 0.05% yesterday, closing at $104.57. The company is attracting investor attention and has closed a $750 million climate private equity fund."}

This approach works fine, however, it consume more tokens that it is necessary

Next, let's try the pydantic approach

## Pydantic Approach

Here we will define a schema to extract ticker first, then feed it to the tool to look up the news

In [186]:
from pydantic import BaseModel, Field

In [187]:
# Define a schema 
class StockTicker(BaseModel):
    ticker: str = Field(
        description = "Stock ticker symbol"
    )

# instantiate a model 
llm_ticker = AzureChatOpenAI(azure_deployment='gpt-4o-mini',
                      temperature= 0)

# structured output 
structured_llm_ticker = llm_ticker.with_structured_output(StockTicker)


# instantiate yahoo news tool 

tool = YahooFinanceNewsTool()

def stock_news(comp_name):
    try: 
        ticker = structured_llm_ticker.invoke(f"What is the stock ticker symbol for the company regarded in this sentence: {comp_name}?")
        news = tool.invoke(ticker.ticker)
        return [Document(page_content=news, metadata={"source": f'https://finance.yahoo.com/quote/{ticker.ticker}/news/'})]
    except Exception as e: 
        return [f"An error occurred during the financial news analysis: {str(e)}"]

In [188]:
stock_news('What happend with Morgan stanley stock yesterday')

[Document(metadata={'source': 'https://finance.yahoo.com/quote/MS/news/'}, page_content="Morgan Stanley (MS) Ascends While Market Falls: Some Facts to Note\nMorgan Stanley (MS) concluded the recent trading session at $104.57, signifying a +0.05% move from its prior day's close.\n\nMorgan Stanley (MS) Could Be a Great Choice\nDividends are one of the best benefits to being a shareholder, but finding a great dividend stock is no easy task. Does Morgan Stanley (MS) have what it takes? Let's find out.\n\nMorgan Stanley (MS) is Attracting Investor Attention: Here is What You Should Know\nMorgan Stanley (MS) has been one of the stocks most watched by Zacks.com users lately. So, it is worth exploring what lies ahead for the stock.\n\nMorgan Stanley Investment Management Closes 1GT Climate Private Equity Fund at $750 Million\nNEW YORK, September 30, 2024--Morgan Stanley Investment Management (MSIM) announced today that it has held the final close of the 1GT climate private equity fund (1GT) at

Since the tool we use YahooFinanceNewsTools only focus on the stock ticker, whatever we put in other than the company name or the ticker does not really matter

In [189]:
stock_news('Morgan Stanley')

[Document(metadata={'source': 'https://finance.yahoo.com/quote/MS/news/'}, page_content="Morgan Stanley (MS) Ascends While Market Falls: Some Facts to Note\nMorgan Stanley (MS) concluded the recent trading session at $104.57, signifying a +0.05% move from its prior day's close.\n\nMorgan Stanley (MS) Could Be a Great Choice\nDividends are one of the best benefits to being a shareholder, but finding a great dividend stock is no easy task. Does Morgan Stanley (MS) have what it takes? Let's find out.\n\nMorgan Stanley (MS) is Attracting Investor Attention: Here is What You Should Know\nMorgan Stanley (MS) has been one of the stocks most watched by Zacks.com users lately. So, it is worth exploring what lies ahead for the stock.\n\nMorgan Stanley Investment Management Closes 1GT Climate Private Equity Fund at $750 Million\nNEW YORK, September 30, 2024--Morgan Stanley Investment Management (MSIM) announced today that it has held the final close of the 1GT climate private equity fund (1GT) at

# Financial Research

Here we will dive further into the company financial side other beyond recent news

In [195]:
def yf_fundamental_analysis(ticker: str):
    """
    Perform a comprehensive fundamental analysis on the given stock symbol.
    
    Args:
        ticker (str): The stock symbol to analyze.
    
    Returns:
        dict: A dictionary with the detailed fundamental analysis results.
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        
        # Data processing
        financials = stock.financials.infer_objects(copy=False)
        balance_sheet = stock.balance_sheet.infer_objects(copy=False)
        cash_flow = stock.cashflow.infer_objects(copy=False)

        # Fill missing values
        financials = financials.ffill()
        balance_sheet = balance_sheet.ffill()
        cash_flow = cash_flow.ffill()

        # Key Ratios and Metrics
        ratios = {
            "P/E Ratio": info.get('trailingPE'),
            "Forward P/E": info.get('forwardPE'),
            "P/B Ratio": info.get('priceToBook'),
            "P/S Ratio": info.get('priceToSalesTrailing12Months'),
            "PEG Ratio": info.get('pegRatio'),
            "Debt to Equity": info.get('debtToEquity'),
            "Current Ratio": info.get('currentRatio'),
            "Quick Ratio": info.get('quickRatio'),
            "ROE": info.get('returnOnEquity'),
            "ROA": info.get('returnOnAssets'),
            "ROIC": info.get('returnOnCapital'),
            "Gross Margin": info.get('grossMargins'),
            "Operating Margin": info.get('operatingMargins'),
            "Net Profit Margin": info.get('profitMargins'),
            "Dividend Yield": info.get('dividendYield'),
            "Payout Ratio": info.get('payoutRatio'),
        }

        # Growth Rates
        revenue = financials.loc['Total Revenue'] if 'Total Revenue' in financials.index else pd.Series()
        net_income = financials.loc['Net Income'] if 'Net Income' in financials.index else pd.Series()
        revenue_growth = revenue.pct_change(periods=-1).iloc[0] if len(revenue) > 1 else None
        net_income_growth = net_income.pct_change(periods=-1, fill_method=None).iloc[0] if len(net_income) > 1 else None

        growth_rates = {
            "Revenue Growth (YoY)": revenue_growth,
            "Net Income Growth (YoY)": net_income_growth,
        }

        # Valuation
        market_cap = info.get('marketCap')
        enterprise_value = info.get('enterpriseValue')

        valuation = {
            "Market Cap": market_cap,
            "Enterprise Value": enterprise_value,
            "EV/EBITDA": info.get('enterpriseToEbitda'),
            "EV/Revenue": info.get('enterpriseToRevenue'),
        }

        # Future Estimates
        estimates = {
            "Next Year EPS Estimate": info.get('forwardEps'),
            "Next Year Revenue Estimate": info.get('revenueEstimates', {}).get('avg'),
            "Long-term Growth Rate": info.get('longTermPotentialGrowthRate'),
        }

        # Simple DCF Valuation (very basic)
        free_cash_flow = cash_flow.loc['Free Cash Flow'].iloc[0] if 'Free Cash Flow' in cash_flow.index else None
        wacc = 0.1  # Assumed Weighted Average Cost of Capital
        growth_rate = info.get('longTermPotentialGrowthRate', 0.03)
        
        def simple_dcf(fcf, growth_rate, wacc, years=5):
            if fcf is None or growth_rate is None:
                return None
            terminal_value = fcf * (1 + growth_rate) / (wacc - growth_rate)
            dcf_value = sum([fcf * (1 + growth_rate) ** i / (1 + wacc) ** i for i in range(1, years + 1)])
            dcf_value += terminal_value / (1 + wacc) ** years
            return dcf_value

        dcf_value = simple_dcf(free_cash_flow, growth_rate, wacc)

        # Prepare the results
        analysis = {
            "Company Name": info.get('longName'),
            "Sector": info.get('sector'),
            "Industry": info.get('industry'),
            "Key Ratios": ratios,
            "Growth Rates": growth_rates,
            "Valuation Metrics": valuation,
            "Future Estimates": estimates,
            "Simple DCF Valuation": dcf_value,
            "Last Updated": datetime.fromtimestamp(info.get('lastFiscalYearEnd', 0)).strftime('%Y-%m-%d'),
            "Data Retrieval Date": datetime.now().strftime('%Y-%m-%d'),
        }

        # Add interpretations
        interpretations = {}
        if ratios.get('P/E Ratio') is not None:
            interpretations["P/E Ratio"] = "High P/E might indicate overvaluation or high growth expectations" if ratios['P/E Ratio'] > 20 else "Low P/E might indicate undervaluation or low growth expectations"
        
        if ratios.get('Debt to Equity') is not None:
            interpretations["Debt to Equity"] = "High leverage" if ratios['Debt to Equity'] > 2 else "Conservative capital structure"
        
        if ratios.get('ROE') is not None:
            interpretations["ROE"] = "Strong returns" if ratios['ROE'] > 0.15 else "Potential profitability issues"
        
        if growth_rates.get('Revenue Growth (YoY)') is not None:
            interpretations["Revenue Growth"] = "Strong growth" if growth_rates['Revenue Growth (YoY)'] > 0.1 else "Slowing growth"

        analysis["Interpretations"] = interpretations

        return analysis

    except Exception as e:
        return f"An error occurred during the analysis: {str(e)}"
    

def stock_financial(comp_name):
    try: 
        ticker = structured_llm_ticker.invoke(f"What is the stock ticker symbol for the company mentioned in the sentence: {comp_name}?")
        financials = yf_fundamental_analysis(ticker.ticker)

        # convert the diction to a json string 
        financials_str = json.dumps(financials, indent = 2)

        return [Document(page_content=financials_str, metadata={'source': f"https://finance.yahoo.com/quote/{ticker.ticker}/financials/"})]

    except Exception as e: 
        return [f"An error occured during the financial research analysis: {str(e)}"]

In [196]:
stock_financial('intel')

[Document(metadata={'source': 'https://finance.yahoo.com/quote/INTC/financials/'}, page_content='{\n  "Company Name": "Intel Corporation",\n  "Sector": "Technology",\n  "Industry": "Semiconductors",\n  "Key Ratios": {\n    "P/E Ratio": 93.35417,\n    "Forward P/E": 20.004465,\n    "P/B Ratio": 0.8314161,\n    "P/S Ratio": 1.7380633,\n    "PEG Ratio": 5.54,\n    "Debt to Equity": 44.032,\n    "Current Ratio": 1.587,\n    "Quick Ratio": 1.079,\n    "ROE": 0.00782,\n    "ROA": 0.00162,\n    "ROIC": null,\n    "Gross Margin": 0.41416,\n    "Operating Margin": -0.07956,\n    "Net Profit Margin": 0.017690001,\n    "Dividend Yield": 0.0221,\n    "Payout Ratio": 2.0833\n  },\n  "Growth Rates": {\n    "Revenue Growth (YoY)": -0.13997525930155108,\n    "Net Income Growth (YoY)": -0.789243823309209\n  },\n  "Valuation Metrics": {\n    "Market Cap": 95803785216,\n    "Enterprise Value": 125555843072,\n    "EV/EBITDA": 11.857,\n    "EV/Revenue": 2.278\n  },\n  "Future Estimates": {\n    "Next Year 